# Test

#### Import the necessary libraries

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import torch

# Add the root project directory to the Python path
ROOT = Path.cwd().parent  # This will get the project root since the notebook is in 'notebooks/'
sys.path.append(str(ROOT))
from configs.path_config import EXTRACTED_DATA_DIR, WEIGHTS_DIR, FEATURES
from src.processing import dataset
from src import utils

#### Create the data loader

In [ ]:
folder_path = EXTRACTED_DATA_DIR / 'group_alvbrodel'
data = dataset.StrainDataset(folder_path, FEATURES, sequence_length=128, start_idx=11112, test_size=0.3)

#### Load the model

In [ ]:
model = torch.load(WEIGHTS_DIR / 'lstm_model_4_54_1_5_0.01_0.3.pth')
print(model)

#### Plot the anomalies

In [ ]:
N = len(data.test_data)

feature_names = data.feature_names
timestamps = pd.DatetimeIndex(np.concatenate(data.timestamps_test))[:N]

reconstructed = utils.plot_reconstruction(data.test_data, model, N, feature_names, timestamps, mode=3)